<a href="https://colab.research.google.com/github/nabeelsana/Nabeel_NLP_Projects/blob/main/Voice_to_text_using_Tensorflow_Using_Spectrogram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
from IPython.display import Audio

In [ ]:
import numpy as np
import sklearn
import csv
import os
import pandas as pd

In [ ]:
import tensorflow as tf
tf.__version__

'2.12.0'

In [ ]:
!pip install jiwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 39.1 MB/s eta 0:00:00


In [ ]:
from jiwer import wer

# DATASET LINK

https://keithito.com/LJ-Speech-Dataset/

In [ ]:
#Mounting Google drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
# STEP 1: Listing all audio clips and verifying their presence, then extract file names without extension

In [ ]:
#files = os.listdir('/gdrive/MyDrive/data/Speech2text/LJSpeech-1.1/wavs')

print(type(files))
print(files[:10])


<class 'list'>
['LJ050-0150.wav', 'LJ007-0088.wav', 'LJ002-0275.wav', 'LJ016-0071.wav', 'LJ015-0012.wav', 'LJ018-0279.wav', 'LJ011-0263.wav', 'LJ046-0108.wav', 'LJ011-0242.wav', 'LJ002-0085.wav']


In [ ]:
#storing file names in drive as a csv file

with open('/gdrive/MyDrive/data/Speech2text/LJSpeech-1.1/LJclipnames.csv', "w", newline = '') as file:
  writer = csv.writer(file)
  for name in files:
    writer.writerow([name])         #[name] will print each name as a new row

In [ ]:
#Importing file as a dataframe

clips_df = pd.read_csv('/gdrive/MyDrive/data/Speech2text/LJSpeech-1.1/LJclipnames.csv')

In [ ]:
clips_df.columns = (['Clips_name'])
print(len(clips_df))
print(clips_df.head())

13099
       Clips_name
0  LJ007-0088.wav
1  LJ002-0275.wav
2  LJ016-0071.wav
3  LJ015-0012.wav
4  LJ018-0279.wav


In [ ]:
clips_df['file_name']= clips_df['Clips_name']

KeyError: ignored

In [ ]:
clips_df.head()

,LJ050-0150.wav
0,LJ007-0088.wav
1,LJ002-0275.wav
2,LJ016-0071.wav
3,LJ015-0012.wav
4,LJ018-0279.wav


In [ ]:
clips_df[['file_name', 'Extension']] = clips_df['file_name'].str.split('.',1, expand = True)

In [ ]:
print(clips_df.head())
print(len(clips_df))

       Clips_name   file_name Extension
0  LJ007-0088.wav  LJ007-0088       wav
1  LJ002-0275.wav  LJ002-0275       wav
2  LJ016-0071.wav  LJ016-0071       wav
3  LJ015-0012.wav  LJ015-0012       wav
4  LJ018-0279.wav  LJ018-0279       wav
13099


In [ ]:
#STEP 2: Read Meta data file contains clips names and sentences

In [ ]:
meta_df = pd.read_csv('/gdrive/MyDrive/data/Speech2text/LJSpeech-1.1/metadata.csv',sep = '|', header = None, )  #make sure file is UTF8 encoded



In [ ]:
meta_df.columns = (['file_name', 'raw_transcription', 'transcription'])

In [ ]:
meta_df.head(5)

,file_name,raw_transcription,transcription
0,LJ001-0001,"Printing, in the only sense with which we are ...","Printing, in the only sense with which we are ..."
1,LJ001-0002,in being comparatively modern.,in being comparatively modern.
2,LJ001-0003,For although the Chinese took impressions from...,For although the Chinese took impressions from...
3,LJ001-0004,"produced the block books, which were the immed...","produced the block books, which were the immed..."
4,LJ001-0005,the invention of movable metal letters in the ...,the invention of movable metal letters in the ...


In [ ]:
meta_df[meta_df['transcription'].isnull()]   # To check if any column in Trabscription is Null , will drop these lines

,file_name,raw_transcription,transcription
945,LJ004-0076,"Disease, cold, famine, nakedness, and contagio...",NaN
948,LJ004-0079,All measures and practices in prison which may...,NaN
1119,LJ004-0250,It is obvious that reformation must be materia...,NaN
1147,LJ005-0028,"There should be no tea and sugar, no assemblag...",NaN
1149,LJ005-0030,"In prisons, which are really meant to keep the...",NaN
1196,LJ005-0077,expedient to introduce such measures and arran...,NaN
1371,LJ005-0253,to inquire into and report upon the several ja...,NaN
1400,LJ005-0282,"the comparatively innocent are seduced, the un...",NaN
1500,LJ006-0083,"So great is the authority exercised by him,|So...",NaN
1655,LJ006-0238,There was a disturbance in the transport yard ...,NaN


In [ ]:
meta_df.dropna(inplace= True)  # drop those lines where transcription column is Null

In [ ]:
len(meta_df)

13084

In [ ]:
#STEP 3: Compare clips df with Meta df and drop lines in meta file that are not present as clip

In [ ]:
metadata_df = pd.merge(clips_df, meta_df, on = 'file_name')

KeyError: ignored

In [ ]:
print(len(metadata_df))
print(metadata_df.head(1))
print(metadata_df.columns)

NameError: ignored

In [ ]:
#Just for comfort write file as xls

In [ ]:
#metadata_df.to_excel('/gdrive/MyDrive/data/Speech2text/LJSpeech-1.1/df.xlsx')


In [ ]:
metadata_df = pd.read_excel('/gdrive/MyDrive/data/Speech2text/LJSpeech-1.1/df.xlsx')

In [ ]:
len(metadata_df)

13083

In [ ]:
#Step 4: Get metadata working file

In [ ]:
meta_df = metadata_df[['Clips_name', 'file_name', 'transcription']]

In [ ]:
meta_df = meta_df.sample(frac= 1).reset_index(drop= True)

In [ ]:
meta_df.head(5)

,Clips_name,file_name,transcription
0,LJ003-0116.wav,LJ003-0116,"A jury having been selected and duly sworn, th..."
1,LJ007-0207.wav,LJ007-0207,but this had the effect of throwing them into ...
2,LJ001-0101.wav,LJ001-0101,It is discouraging to note that the improvemen...
3,LJ044-0194.wav,LJ044-0194,Oswald's attempt to go to Cuba
4,LJ002-0010.wav,LJ002-0010,these numbers had risen to two hundred seventy...


In [ ]:
#meta_df = meta_df[:100]   # initial testing of model with 100 examples

In [ ]:
len(meta_df)

13083

In [ ]:
#Step 5: Split into Training Validation data set

In [ ]:
split =int(len(meta_df)*0.9)
print(split)

11774


In [ ]:
df_train = meta_df[:split]
df_val = meta_df[split:]
print(len(df_train))
print(len(df_val))

11774
1309


In [ ]:
#MACHINE LEARNING PROCESS

In [ ]:
path = '/gdrive/MyDrive/data/Speech2text/LJSpeech-1.1/wavs/'

In [ ]:
#Functions to process input sentence

In [ ]:
characters = [x for x in "abcdefghijklmnopqrstuvwxyz'?! "]

In [ ]:
print(len(characters))

30


In [ ]:
char_to_num = tf.keras.layers.StringLookup(vocabulary = characters, oov_token= "")

In [ ]:
print(char_to_num.get_vocabulary())

['', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'", '?', '!', ' ']


In [ ]:
print(char_to_num.vocabulary_size())

31


In [ ]:
output_dim = char_to_num.vocabulary_size()+1

In [ ]:
print(output_dim)

32


In [ ]:
num_to_char = tf.keras.layers.StringLookup(vocabulary = char_to_num.get_vocabulary(), oov_token= "", invert = True)

In [ ]:
#Function to fetch audio files, create spectrogram and process corresponding sentences

In [ ]:
frame_length = 256
frame_step = 160
fft_length = 384

# frame length   = no of data points in each sliding window here 256 points will be covered in each frame out of 212893 input data point
# frame steps/hop size = no of data points to skip when sliding window so first 256 data points then skip 160 then start from data point 416 to 672, then skip another 160 and so on
#212893/frame steps = no of row  here 212893/160 =1329
#the size of the FFT (here 384) determines the number of frequency bins used to represent the spectrum of each frame of the signal.
#The number of frequency bins in the resulting spectrum is equal to half the FFT size plus one. so 384/2+1 = 193
#This means each row with contain  193 data points i.e columns

In [ ]:
def encode_single_sample(audio_file, label):
  #process audio generating spectrogram
  #1 reading and decoding file and change datatype to float
  file = tf.io.read_file(path+audio_file)
  audio,_ = tf.audio.decode_wav(file)
  audio = tf.squeeze(audio, axis = -1)
  audio = tf.cast(audio, tf.float32)

  #2 Getting Spectrograph
  spectrograph = tf.signal.stft(audio, frame_length = frame_length, frame_step = frame_step, fft_length= fft_length)

  #3 Getting magnitude  # This does not change shape of output which will have 193 data points in each row.
  spectrograph = tf.abs(spectrograph)
  spectrograph = tf.math.pow(spectrograph, 0.5)

  #4 Normalization

  means = tf.math.reduce_mean(spectrograph, 1, keepdims = True)
  stddev = tf.math.reduce_std(spectrograph, 1, keepdims = True)
  spectrogram = (spectrograph - means)/(stddev+ 1e-10)

  #process sentence

  label = tf.strings.lower(label)
  label = tf.strings.unicode_split(label,input_encoding = "UTF-8")
  label = char_to_num(label)

  return spectrogram, label




In [ ]:
#print(spectrograph.shape)  # Spectrograph of one voice clip # 193 frequencies with 1329 times stamps
#(1329, 193)

#The spectrogram is a spectro-temporal representation of the sound.
#The horizontal direction of the spectrogram represents time, the vertical direction represents frequency.
#The time scale of the spectrogram is the same as that of the waveform, so the spectrogram reacts to your zooming and scrolling.



In [ ]:
#Creating Dataset Object   # If any of column has null value , will give error.


In [ ]:
batch_size = 32

train_dataset = tf.data.Dataset.from_tensor_slices((list(df_train['Clips_name']), list(df_train['transcription'])))

train_dataset = train_dataset.map(encode_single_sample, num_parallel_calls = tf.data.AUTOTUNE).padded_batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
train_dataset.element_spec

(TensorSpec(shape=(None, None, 193), dtype=tf.float32, name=None),
 TensorSpec(shape=(None, None), dtype=tf.int64, name=None))

In [ ]:
for x,  y in train_dataset:
  print(x.shape, y.shape)


(32, 1369, 193) (32, 160)
(32, 1339, 193) (32, 175)
(32, 1376, 193) (32, 149)
(32, 1387, 193) (32, 164)
(32, 1373, 193) (32, 153)
(32, 1371, 193) (32, 149)
(32, 1352, 193) (32, 165)
(32, 1369, 193) (32, 145)
(32, 1353, 193) (32, 151)
(32, 1377, 193) (32, 142)
(32, 1376, 193) (32, 172)
(32, 1333, 193) (32, 150)
(32, 1385, 193) (32, 172)
(32, 1390, 193) (32, 183)
(32, 1323, 193) (32, 169)
(32, 1385, 193) (32, 165)
(32, 1390, 193) (32, 152)
(32, 1381, 193) (32, 158)
(32, 1376, 193) (32, 157)
(32, 1363, 193) (32, 153)
(32, 1307, 193) (32, 153)
(32, 1389, 193) (32, 157)
(32, 1381, 193) (32, 182)
(32, 1379, 193) (32, 157)
(32, 1240, 193) (32, 139)
(32, 1390, 193) (32, 154)
(32, 1389, 193) (32, 156)
(32, 1345, 193) (32, 161)
(32, 1310, 193) (32, 158)
(32, 1345, 193) (32, 146)
(32, 1387, 193) (32, 157)
(32, 1381, 193) (32, 160)
(32, 1329, 193) (32, 151)
(32, 1376, 193) (32, 154)
(32, 1326, 193) (32, 157)
(32, 1389, 193) (32, 156)


KeyboardInterrupt: ignored

In [ ]:
y[12]

<tf.Tensor: shape=(156,), dtype=int64, numpy=
array([14, 15, 23,  0, 30, 20,  8,  9, 19, 30,  9, 19, 30, 21, 14,  4, 15,
       21,  2, 20,  5,  4, 12, 25, 30, 20,  8,  5, 30, 15, 18,  4,  5, 18,
       30, 15,  6, 30, 19, 21,  3,  3,  5, 19, 19,  9, 15, 14, 30, 15,  6,
       30,  6, 15, 18, 13, 19, 30,  9, 14, 30,  7,  5, 15, 12, 15,  7,  9,
        3,  1, 12, 30, 20,  9, 13,  5, 19, 30,  0,  0, 30,  9,  0,  5,  0,
        0, 30,  9, 14, 30, 20,  8,  5, 30, 16,  8, 25, 12, 15,  7,  5, 14,
        9,  3, 30, 19,  5, 18,  9,  5, 19,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0])>

In [ ]:
val_dataset = tf.data.Dataset.from_tensor_slices((list(df_val['Clips_name']), list(df_val['transcription'])))
val_dataset = val_dataset.map(encode_single_sample, num_parallel_calls = tf.data.AUTOTUNE).padded_batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
#Machine Learning Model

In [ ]:
from tensorflow.keras.models import Sequential

In [ ]:
from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization, ReLU, Reshape, Dropout, GRU, Reshape, Bidirectional

In [ ]:
model = Sequential([
    Reshape((-1, 193, 1), name="expand_dim", input_shape=(None, 193)),     # see explanation below                    (batch, rows, columns/data point in each row,      channels)
    #                                                                                                                   (-1, -1,         193,                            1) -1 means variable number
    Conv2D(filters = 32, kernel_size = [11,41], strides = 2, padding = 'same', use_bias = False,  name = 'conv_1'), # (-1,    -1,     193/2 = 97,                        32)
    BatchNormalization(name = 'conv_1_bn'),
    ReLU(name= 'conv_1_relu'),
    Conv2D(filters = 32, kernel_size = [11,41], strides = 2, padding = 'same', use_bias = False,  name = 'conv_2'),  # (-1,     -1,     97/2 = 49,                        32)
    BatchNormalization(name = 'conv_2_bn'),
    ReLU(name= 'conv_2_relu'),
    Reshape((-1,49*32)), #   (-1,-1,  columns*filters )
    #The RNN expects that the input is going to be sequential. Therefore, the input has the shape  [batch_size, time, feature_size]
    #reshape as [b_size, num_frames, am_width * am_height * num_filters]
    Bidirectional((GRU(units =512, activation = 'tanh',recurrent_activation='sigmoid', use_bias=True,return_sequences=True, reset_after = True , name = 'GRU_1')), name = 'bidir_1' ),
    Dropout(rate = 0.5, name = 'drop_out_1'),
    Bidirectional((GRU(units =512, activation = 'tanh',recurrent_activation='sigmoid', use_bias=True,return_sequences=True, reset_after = True , name = 'GRU_2')),  name = 'bidir_2' ),
    Dropout(rate = 0.5, name = 'drop_out_2'),
    Bidirectional((GRU(units =512, activation = 'tanh',recurrent_activation='sigmoid', use_bias=True,return_sequences=True, reset_after = True , name = 'GRU_3')),  name = 'bidir_3' ),
    Dropout(rate = 0.5, name = 'drop_out_3'),
    Bidirectional((GRU(units =512, activation = 'tanh',recurrent_activation='sigmoid', use_bias=True,return_sequences=True, reset_after = True , name = 'GRU_4')),  name = 'bidir_4' ),
    Dropout(rate = 0.5, name = 'drop_out_4'),
    Bidirectional((GRU(units =512, activation = 'tanh',recurrent_activation='sigmoid', use_bias=True,return_sequences=True, reset_after = True , name = 'GRU_5')),  name = 'bidir_5' ),
    Dense(units= 512*2, name = 'dense1'),
    ReLU(name= 'Dense_1_relu'),
    Dropout(rate = 0.5, name = 'drop_out_5'),
    Dense(units = output_dim, activation = 'softmax')



])

In [ ]:
"""
input_shape=(None, 193) is a parameter used in defining the shape of the input data for a machine learning model.

In this case, None indicates that the input can have a variable number of rows (the first dimension), and 193 indicates that the input data has 193 columns (the second dimension).
The first dimension is often left as None to allow for variable batch sizes during training or inference.

For example, if we have a dataset of audio recordings, the first dimension of the input might correspond to the number of time steps in the recording,
while the second dimension might correspond to the number of frequency bins in a spectrogram representation of the audio signal.
In this case, input_shape=(None, 193) would be appropriate, as the length of the audio recordings can vary, but the number of frequency bins is fixed at 193.
If the input shape is (None, 193), applying the Reshape layer with Reshape((-1, 193, 1)) will transform the input into a new shape of (None,None, 193, 1).

Here, -1 is a special value that indicates that the size of the first dimension of the input should be inferred from the remaining dimensions and the batch size.
In other words, the number of samples in the batch can vary, but the size of the other dimensions remains constant.

The resulting shape (None,None 193, 1) represents a 4D tensor, where the first dimension is the batch size, the second dimension is the number of time steps (rows), and the third dimension is freq bins and 4th dim is
 the number of channels
(which is 1 in this case). This 4D tensor is commonly used as input for 2D convolutional neural networks (CNNs) that are designed to operate on sequences of data,
 such as time series or audio signals.

"""

'\ninput_shape=(None, 193) is a parameter used in defining the shape of the input data for a machine learning model.\n\nIn this case, None indicates that the input can have a variable number of rows (the first dimension), and 193 indicates that the input data has 193 columns (the second dimension). The first dimension is often left as None to allow for variable batch sizes during training or inference.\n\nFor example, if we have a dataset of audio recordings, the first dimension of the input might correspond to the number of time steps in the recording, while the second dimension might correspond to the number of frequency bins in a spectrogram representation of the audio signal. In this case, input_shape=(None, 193) would be appropriate, as the length of the audio recordings can vary, but the number of frequency bins is fixed at 193.\nIf the input shape is (None, 193), applying the Reshape layer with Reshape((-1, 193, 1)) will transform the input into a new shape of (None, 193, 1).\n\

In [ ]:
 #Convolution output:
    #The number of filters (32) corresponds to the number of output channels.
    #The kernel size of [11, 41] indicates that the convolution will be done over an 11x41 window.
    #The stride of 2 indicates that the filter will move 2 pixels at a time.
    #The padding of 'same' means that the input will be padded with zeros to ensure that the output has the same spatial dimensions as the input.
    #With these parameters, the convolution operation will reduce the spatial dimensions of the input by a factor of 2 in both the vertical and horizontal directions
    #(since the stride is 2), while increasing the number of channels to 32.
    #output will be of shape (-1,-1,  columns/2 rounded to nearest whole,filters )

In [ ]:
"""
explanation for first reshape
The Reshape layer is used to reshape the input tensor into a new shape. In this case, the layer will reshape the input tensor to have the following dimensions:

The first dimension will be set to -1, which means that it will be dynamically computed based on the size of the input tensor and the other dimensions specified in the reshape.
The second dimension will be set to 1, indicating that each element in the input tensor will be reshaped into a 1-dimensional vector.
The third dimension will be set to input_dim, which is the size of the input data dimension.
The fourth dimension will be set to 1, indicating that each element in the reshaped tensor will have a single channel.
Overall, this layer will transform the input tensor into a 4-dimensional tensor with a variable number of rows, a single column, input_dim channels, and a single batch dimension. This can be useful, for example, when preparing image data for input to a convolutional neural network.*/
"""


'\nexplanation for reshape\nThe Reshape layer is used to reshape the input tensor into a new shape. In this case, the layer will reshape the input tensor to have the following dimensions:\n\nThe first dimension will be set to -1, which means that it will be dynamically computed based on the size of the input tensor and the other dimensions specified in the reshape.\nThe second dimension will be set to 1, indicating that each element in the input tensor will be reshaped into a 1-dimensional vector.\nThe third dimension will be set to input_dim, which is the size of the input data dimension.\nThe fourth dimension will be set to 1, indicating that each element in the reshaped tensor will have a single channel.\nOverall, this layer will transform the input tensor into a 4-dimensional tensor with a variable number of rows, a single column, input_dim channels, and a single batch dimension. This can be useful, for example, when preparing image data for input to a convolutional neural network.

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate = 1e-4)

In [ ]:
#Defining CTC Loss Function
def CTCLoss(y_true, y_pred):
  batch_length = tf.cast(tf.shape(y_true)[0], dtype= 'int64')
  input_length = tf.cast(tf.shape(y_pred)[1], dtype= 'int64' )
  input_length = input_length*tf.ones(shape = (batch_length,1),dtype= 'int64' )

  label_length = tf.cast(tf.shape(y_true)[1], dtype= 'int64' )
  label_length = label_length*tf.ones(shape = (batch_length,1),dtype= 'int64' )




  loss = tf.keras.backend.ctc_batch_cost(y_true,y_pred, input_length, label_length)  #https://keras.io/ko/backend/  for ctc_batch_cost
  return loss


In [ ]:
model.compile(optimizer = opt, loss = CTCLoss)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 expand_dim (Reshape)        (None, None, 193, 1)      0         
                                                                 
 conv_1 (Conv2D)             (None, None, 97, 32)      14432     
                                                                 
 conv_1_bn (BatchNormalizati  (None, None, 97, 32)     128       
 on)                                                             
                                                                 
 conv_1_relu (ReLU)          (None, None, 97, 32)      0         
                                                                 
 conv_2 (Conv2D)             (None, None, 49, 32)      461824    
                                                                 
 conv_2_bn (BatchNormalizati  (None, None, 49, 32)     128       
 on)                                                    

In [ ]:
hitory = model.fit(train_dataset, validation_data = val_dataset ,epochs = 10)

Epoch 1/10
368/368 [==============================] - 2909s 8s/step - loss: inf - val_loss: 420.1360
Epoch 2/10
368/368 [==============================] - 1086s 3s/step - loss: inf - val_loss: 308.1458
Epoch 3/10
368/368 [==============================] - 1092s 3s/step - loss: inf - val_loss: 310.2918
Epoch 4/10
368/368 [==============================] - 1083s 3s/step - loss: inf - val_loss: 288.8520
Epoch 5/10
368/368 [==============================] - 1140s 3s/step - loss: inf - val_loss: 340.5425
Epoch 6/10
368/368 [==============================] - 1092s 3s/step - loss: inf - val_loss: 304.8803
Epoch 7/10
368/368 [==============================] - 1097s 3s/step - loss: inf - val_loss: 277.8847
Epoch 8/10
368/368 [==============================] - 1086s 3s/step - loss: inf - val_loss: 212.0180
Epoch 9/10
368/368 [==============================] - 1076s 3s/step - loss: inf - val_loss: 162.4342
Epoch 10/10
368/368 [==============================] - 1085s 3s/step - loss: inf - val_loss

In [ ]:
#Defining Prediction decoding function

def decode_batch_prediction(pred):
  input_length = np.ones(pred.shape[0])*pred.shape[1]
  results = tf.keras.backend.ctc_decode(pred, input_length = input_length, greedy = True)[0][0]                     # https://keras.io/ko/backend/ (for CTC_decode)
  output_text = []
  for result in results:
    result = tf.strings.reduce_join(num_to_char(result)).numpy().decode("utf-8")
    output_text.append(result)
  return output_text



In [ ]:
for batch in val_dataset:
  x,y = batch
  break

In [ ]:
print(x.shape, y.shape)

(32, 1326, 193) (32, 145)


In [ ]:
prediction  = model.predict(x)

1/1 [==============================] - 1s 634ms/step


In [ ]:
prediction[0]

array([[1.6975449e-05, 2.6481348e-07, 7.5614088e-07, ..., 3.2929581e-09,
        2.9511993e-06, 6.6945344e-05],
       [1.3540813e-04, 1.2793596e-04, 1.1477180e-05, ..., 1.3332541e-07,
        3.0134765e-05, 1.5672807e-02],
       [1.5698040e-05, 1.3331220e-05, 1.7497338e-07, ..., 2.2195399e-08,
        7.2499772e-04, 2.0404821e-02],
       ...,
       [5.5278075e-01, 6.1588999e-08, 4.3685451e-09, ..., 5.6651611e-08,
        3.9144695e-08, 4.4721004e-01],
       [1.7801176e-01, 1.2674900e-07, 1.2083365e-08, ..., 1.1423799e-07,
        1.0476531e-07, 8.2197189e-01],
       [9.7543937e-01, 4.6022160e-07, 4.8716359e-08, ..., 2.7148698e-07,
        6.5562908e-08, 2.4529876e-02]], dtype=float32)

In [ ]:
prediction.shape[0], prediction.shape[1]

(32, 332)

In [ ]:
prediction_decoded = decode_batch_prediction(prediction)

In [ ]:
prediction_decoded [0], prediction_decoded[1] , prediction_decoded[2]

('to ivie the feld ofice if the sutic displae sie f in cree ne as s s ',
 'mrs pane ponted out that most of the oswaldsin r r ',
 'but mstrpeares son haridly considered the converisfi')

In [ ]:
y[0], y[1], y[2]

(<tf.Tensor: shape=(145,), dtype=int64, numpy=
 array([20, 15, 30,  1,  4, 22,  9, 19,  5, 30, 20,  8,  5, 30,  6,  9,  5,
        12,  4, 30, 15,  6,  6,  9,  3,  5, 30,  9,  6, 30, 20,  8,  5, 30,
        19, 21,  2, 10,  5,  3, 20, 30,  4,  9, 19, 16, 12,  1, 25, 19, 30,
        19,  9,  7, 14, 19, 30, 15,  6, 30,  9, 14,  3, 18,  5,  1, 19,  5,
         4, 30,  4,  1, 14,  7,  5, 18, 30, 15, 18, 30, 16, 12,  1, 14, 19,
        30, 20, 15, 30, 12,  5,  1, 22,  5, 30,  8,  9, 19, 30,  8, 15, 13,
         5, 30,  1, 18,  5,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0])>,
 <tf.Tensor: shape=(145,), dtype=int64, numpy=
 array([13, 18, 19,  0, 30, 16,  1,  9, 14,  5, 30, 16, 15,  9, 14, 20,  5,
         4, 30, 15, 21, 20, 30, 20,  8,  1, 20, 30, 13, 15, 19, 20, 30, 15,
         6, 30, 20,  8,  5, 30, 15, 19, 23,  1, 12,  4, 19, 27, 30, 16, 15,
        19, 19,  5, 19,

In [ ]:
target = []

In [ ]:
target[0], target[1], target[2]

('to advise the field office if the subject displays signs of increased danger or plans to leave his home area',
 "mrs paine pointed out that most of the oswalds' possessions were in the garage",
 'but mr pearson hardly considered the converse sufficiently and')

In [ ]:
for label in y:
  label = tf.strings.reduce_join(num_to_char(label)).numpy().decode("utf-8")
  target.append(label)

In [ ]:
#Inferencing

predictions = []
targets = []

for batch in val_dataset:
  x,y = batch
  batch_predictions = model.predict(x)
  batch_predictions = decode_batch_prediction(batch_predictions)
  predictions.extend(batch_predictions)

  for label in y:
    label = tf.strings.reduce_join(num_to_char(label)).numpy().decode("utf-8")
    targets.append(label)

wer_score = wer(targets, predictions)
print(f"WER: {wer_score:.4f}")

for i in np.random.randint(0,len(predictions),5):
  print(f"Target :{targets[i]}")
  print(f"Predictions :{predictions[i]}")






1/1 [==============================] - 3s 3s/step
WER: 0.7651
Target :and any other series of organs of vertebrates would give the same evidence of fundamental resemblances homologies
Predictions :n ay other sceares of orgins of firtaberits woun give the same evides o l ls s
Target :the city justices had not fulfilled this obligation
Predictions :the sity justics and no fof i i
Target :and this particular official took excellent care to select as residents for his own ward those most suitable from his own point of view
Predictions :and this prticila ficial tok exullit car to sullect has reidentc for his on word tos os i 
Target :serving only coffee fruit juices and no hard liquors or beer end quote
Predictions :serving only cof fro jose nd r r r o 
Target :further sums were to have been paid after the escape
Predictions :for ther sume oe e a 


In [ ]:
predictions

['to ivie the feld ofice if the sutic displae sie f in cree ne as s s ',
 'mrs pane ponted out that most of the oswaldsin r r ',
 'but mstrpeares son haridly considered the converisfi',
 'tat he goudichanse of acompliong r',
 'the files ar mandaly maintaned with houd ous a o i i i ici',
 'owiththa calfication to be considerd presiply the n serc to this custon asd odti ivif',
 'vobal sid that oswald quot wotin star any fie b iffu otd docoe won w wi ws t t e e e',
 'owers sequot nowhe was oswald mad oa o oe ',
 'he was found guilty of an asaled withintent and sentence to trant frctinfofe',
 'arbeerd than eare personl strangke in the acomplichement of such tas a a o o oro',
 'thews asogot suplie a r',
 'in deciding whether oswald cared the assassination wepen in the ba which fraur and mrse rad l',
 'he was waring an sicred jacket which h had noted win in oe f fr ',
 'the cort of the marial s was instituted bi os e i i ri',
 'id wove wa cald from the discuion in cape fre',
 'whene le becam

In [ ]:
len(targets)

1309

In [ ]:
len(predictions)

1309

In [ ]:
targets[0]

'to advise the field office if the subject displays signs of increased danger or plans to leave his home area'

In [ ]:
predictions[0]

'to ivie the feld ofice if the sutic displae sie f in cree ne as s s '

In [ ]:
wer_score1 = wer(targets[0], predictions[0])

In [ ]:
print(wer_score1)

0.8
